In [ ]:
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
# Set seeds
def set_seeds(seed: int=42):
    """Sets random sets for torch operations.

    Args:
        seed (int, optional): Random seed to set. Defaults to 42.
    """
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)



pretrained_vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT

pretrained_vit = torchvision.models.vit_b_16(weights=pretrained_vit_weights).to(device)

# 3. Freeze the base parameters
for parameter in pretrained_vit.parameters():
    parameter.requires_grad = False

# 4. Change the classifier head
class_names = ['downdog','plank','tree','goddess','warrior2']

set_seeds()
pretrained_vit.heads =nn.Sequential(nn.Linear(in_features=768,out_features=512),nn.ReLU(),nn.Dropout(0.5),nn.Linear(in_features=512,out_features=len(class_names))).to(device)



In [ ]:
from torchinfo import summary


summary(model=pretrained_vit,
        input_size=(32, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

E:\Anaconda\Lib\site-packages\torch\nn\modules\activation.py:1196: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  return torch._native_multi_head_attention(


Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
VisionTransformer (VisionTransformer)                        [32, 3, 224, 224]    [32, 5]              768                  Partial
├─Conv2d (conv_proj)                                         [32, 3, 224, 224]    [32, 768, 14, 14]    (590,592)            False
├─Encoder (encoder)                                          [32, 197, 768]       [32, 197, 768]       151,296              False
│    └─Dropout (dropout)                                     [32, 197, 768]       [32, 197, 768]       --                   --
│    └─Sequential (layers)                                   [32, 197, 768]       [32, 197, 768]       --                   False
│    │    └─EncoderBlock (encoder_layer_0)                   [32, 197, 768]       [32, 197, 768]       (7,087,872)          False
│    │    └─EncoderBlock (encoder_layer_1)                   [32, 197, 768]       [32, 

In [ ]:
# Setup directory paths to train and test images
train_dir=r'F:\ML_Projects\Image_Classification\niharika\DATASET\TRAIN'
test_dir =  r'F:\ML_Projects\Image_Classification\niharika\DATASET\TEST'
import numpy as np

In [ ]:
pretrained_vit_transforms = pretrained_vit_weights.transforms()

print(pretrained_vit_transforms)

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)


In [ ]:
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS

):

  # Use ImageFolder to create dataset(s)
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  # Get class names
  class_names = train_data.classes

  # Turn images into data loaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names

In [ ]:
# Setup dataloaders
train_dataloader, test_dataloader, class_names = create_dataloaders(train_dir=train_dir,
                                                                                        test_dir=test_dir,
                                                                                                     transform=pretrained_vit_transforms,
                                                                                                     batch_size=64) # Could increase if we had more samples, such as here: https://arxiv.org/abs/2205.01580 (there are other improvements there too...)                                                                                            batch_size=32) # Could increase if we had more samples, such as here: https://arxiv.org/abs/2205.01580 (there are other improvements there too...)

In [ ]:
import torch
from tqdm.auto import tqdm
from typing import Dict, List, Tuple
from sklearn.metrics import precision_score, recall_score, f1_score

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device,
               lr_scheduler: torch.optim.lr_scheduler = None) -> Tuple[float, float]:



    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(y_pred, dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float, float, float, float]:
    """Tests a PyTorch model for a single epoch.

    Turns a target PyTorch model to "eval" mode and then performs
    a forward pass on a testing dataset.

    Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of testing loss, testing accuracy, precision, recall, and F1-score metrics.
    In the form (test_loss, test_accuracy, precision, recall, f1_score). For example:

    (0.0223, 0.8985, 0.8621, 0.9254, 0.8923)
    """
    # Put model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Lists to store true and predicted labels for computing precision, recall, and F1-score
    true_labels = []
    pred_labels = []

    # Turn off gradient computation
    with torch.no_grad():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # 3. Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

            # Store true and predicted labels
            true_labels.extend(y.cpu().numpy())
            pred_labels.extend(test_pred_labels.cpu().numpy())

    # Compute precision, recall, and F1-score
    precision = precision_score(true_labels, pred_labels, average='weighted')
    recall = recall_score(true_labels, pred_labels, average='weighted')
    f1 = f1_score(true_labels, pred_labels, average='weighted')

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)

    return test_loss, test_acc, precision, recall, f1

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device,
          lr_scheduler: torch.optim.lr_scheduler = None) -> Dict[str, List]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
              train_acc: [...],
              test_loss: [...],
              test_acc: [...]}
    For example if training for epochs=2:
             {train_loss: [2.0616, 1.0537],
              train_acc: [0.3945, 0.3945],
              test_loss: [1.2641, 1.5706],
              test_acc: [0.3400, 0.2973]}
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": [],
               "precision": [],
               "recall": [],
               "f1_score": []
    }


    model.to(device)

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc, precision, recall, f1 = test_step(model=model,
                                                               dataloader=test_dataloader,
                                                               loss_fn=loss_fn,
                                                               device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f} | "
          f"precision: {precision:.4f} | "
          f"recall: {recall:.4f} | "
          f"f1_score: {f1:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        results["precision"].append(precision)
        results["recall"].append(recall)
        results["f1_score"].append(f1)

    # Return the filled results at the end of the epochs
    return results


In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
learning_rate = 0.001

optimizer = torch.optim.Adam(params=pretrained_vit.parameters(),lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

# Train the classifier head of the pretrained ViT feature extractor model
set_seeds()

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
pretrained_vit_results = train(model=pretrained_vit,
                                      train_dataloader=train_dataloader,
                                      test_dataloader=test_dataloader,
                                      optimizer=optimizer,
                                      loss_fn=loss_fn,
                                      epochs=30,
                                      device=device,
                                      lr_scheduler=scheduler


                             )


E:\Anaconda\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0522 | train_acc: 0.6058 | test_loss: 0.4988 | test_acc: 0.7871 | precision: 0.8199 | recall: 0.7681 | f1_score: 0.7609
Epoch: 2 | train_loss: 0.4372 | train_acc: 0.8655 | test_loss: 0.2260 | test_acc: 0.9531 | precision: 0.9500 | recall: 0.9489 | f1_score: 0.9489


KeyboardInterrupt: 

In [ ]:
print(pretrained_vit)

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [ ]:
# # Plot the loss curves
# from helper_functions import plot_loss_curves

# plot_loss_curves(pretrained_vit_results)

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split


In [ ]:
img_height, img_width = 224,224
transform = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor(),
])

In [ ]:
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS

):

  # Use ImageFolder to create dataset(s)
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  # Get class names
  class_names = train_data.classes

  # Turn images into data loaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names

In [ ]:
train_dataloader, test_dataloader, class_names = create_dataloaders(train_dir=train_dir,
                                                                                        test_dir=test_dir,
                                                                                                     transform=transform,
                                                                                                     batch_size=32,num_workers=0) # Could increase if we had more samples, such as here: https://arxiv.org/abs/2205.01580 (there are other improvements there too...)                                                                                            batch_size=32) # Could increase if we had more samples, such as here: https://arxiv.org/abs/2205.01580 (there are other improvements there too...)

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

# Define the ResNet50 model
class ResNet50Model(nn.Module):
    def __init__(self, num_classes):
        super(ResNet50Model, self).__init__()
        # Load the pre-trained ResNet50 model
        self.base_model = models.resnet50(pretrained=True)
        # Freeze the layers
        for param in self.base_model.parameters():
            param.requires_grad = False
        # Modify the classifier
        num_ftrs = self.base_model.fc.in_features
        self.base_model.fc = nn.Sequential(
            nn.Linear(num_ftrs, num_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Forward pass through the base model
        x = self.base_model(x)
        return x

# Instantiate the model
num_classes = 5  # Change this based on your task
model = ResNet50Model(num_classes)

# Print the model architecture
print(model)



E:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
E:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet50Model(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from sklearn.metrics import precision_score, recall_score, f1_score



# Assuming train_dataloader, test_dataloader, and num_classes are defined

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)

# Sample training loop
num_epochs = 50 # Define number of epochs
for epoch in range(num_epochs):
    # Training phase
    model.train()
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluation phase
    model.eval()
    correct = 0
    total = 0
    predicted_labels = []
    true_labels = []
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            predicted_labels.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = correct / total

    # Calculate precision, recall, and F1 score
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')
    f1 = f1_score(true_labels, predicted_labels, average='weighted')

    print(f"Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")


E:\Anaconda\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/50], Accuracy: 0.7553191489361702, Precision: 0.7963943977246055, Recall: 0.7553191489361702, F1 Score: 0.7296188084479487


E:\Anaconda\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [2/50], Accuracy: 0.8319148936170213, Precision: 0.8554446944141273, Recall: 0.8319148936170213, F1 Score: 0.8268994398744705


E:\Anaconda\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

Epoch [48/50], Accuracy: 0.9212765957446809, Precision: 0.9241333636546964, Recall: 0.9212765957446809, F1 Score: 0.9193982196316255


E:\Anaconda\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [49/50], Accuracy: 0.9319148936170213, Precision: 0.9332934729868333, Recall: 0.9319148936170213, F1 Score: 0.9310958632104762


E:\Anaconda\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [50/50], Accuracy: 0.925531914893617, Precision: 0.9268610181572512, Recall: 0.925531914893617, F1 Score: 0.9241036613094546


In [ ]:
import torch

# Assuming `model` is your PyTorch model
torch.save(model.state_dict(), 'model3_resnet.pth')

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split


In [ ]:
img_height, img_width = 224,224
transform = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor(),
])

In [ ]:
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS

):

  # Use ImageFolder to create dataset(s)
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  # Get class names
  class_names = train_data.classes

  # Turn images into data loaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names

In [ ]:
train_dataloader, test_dataloader, class_names = create_dataloaders(train_dir=train_dir,
                                                                                        test_dir=test_dir,
                                                                                                     transform=transform,
                                                                                                     batch_size=32,num_workers=0) # Could increase if we had more samples, such as here: https://arxiv.org/abs/2205.01580 (there are other improvements there too...)                                                                                            batch_size=32) # Could increase if we had more samples, such as here: https://arxiv.org/abs/2205.01580 (there are other improvements there too...)

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

# Define the DenseNet201 model
class DenseNet201Model(nn.Module):
    def __init__(self, num_classes):
        super(DenseNet201Model, self).__init__()
        # Load the pre-trained DenseNet201 model
        self.base_model = models.densenet201(pretrained=True)
        # Freeze the layers
        for param in self.base_model.parameters():
            param.requires_grad = False
        # Modify the classifier
        self.base_model.classifier = nn.Sequential(
            nn.Linear(1920, 512),  # Adjust input size based on DenseNet201 architecture
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Forward pass through the base model
        x = self.base_model(x)
        return x

# Instantiate the model
num_classes = 5  # Change this based on your task
model = DenseNet201Model(num_classes)

# Print the model architecture
print(model)

E:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DenseNet201Model(
  (base_model): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import precision_score, recall_score, f1_score
import torchvision.models as models

criterion = nn.CrossEntropyLoss()  # Assuming you're using CrossEntropyLoss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Sample training loop
num_epochs = 50  # Define number of epochs
for epoch in range(num_epochs):
    # Training phase
    model.train()
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluation phase
    model.eval()
    correct = 0
    total = 0
    predictions = []
    true_labels = []
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            predictions.extend(predicted.tolist())
            true_labels.extend(labels.tolist())

    accuracy = correct / total
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')

    print(f"Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")


E:\Anaconda\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/50], Accuracy: 0.7510638297872341, Precision: 0.8564363136473936, Recall: 0.7067590814448068, F1 Score: 0.7152011921058004


E:\Anaconda\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [2/50], Accuracy: 0.9148936170212766, Precision: 0.9148180431315442, Recall: 0.920308733484388, F1 Score: 0.9136933125556087


E:\Anaconda\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

Epoch [48/50], Accuracy: 0.9638297872340426, Precision: 0.9631367637121377, Recall: 0.9652191287076597, F1 Score: 0.9635092810664444


E:\Anaconda\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [49/50], Accuracy: 0.9574468085106383, Precision: 0.9593842174101397, Recall: 0.9560498028201447, F1 Score: 0.957608017750645


E:\Anaconda\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [50/50], Accuracy: 0.9574468085106383, Precision: 0.9595409496010475, Recall: 0.957512041884099, F1 Score: 0.958249796565194


In [ ]:
import torch
# Assuming `model` is your PyTorch model
torch.save(model.state_dict(), 'model3_densenet.pth')

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models


m1_state_dict = torch.load('model_ViT.pth')

m2_state_dict = torch.load('model3_resnet.pth')
m3_state_dict = torch.load('model3_densenet.pth')

# Define the models architecture
m1 = to_device(pretrained_vit,device)
num_classes = 5

m2= to_device(ResNet50Model(num_classes),device)

m3 =to_device(DenseNet201Model(num_classes),device)


m1.load_state_dict(m1_state_dict)
m2.load_state_dict(m2_state_dict)
m3.load_state_dict(m3_state_dict)



<All keys matched successfully>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models



# Define the ensemble model
class EnsembleModel(nn.Module):
    def __init__(self, m1, m2, m3):
        super(EnsembleModel, self).__init__()
        self.m1 = m1
        self.m2 = m2
        self.m3 = m3
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))  # Adaptive pooling to get a fixed-size output

    def forward(self, x):
        x1 = self.m1(x.clone())  # Clone the input tensor to avoid in-place modifications
        x2 = self.m2(x.clone())
        x3 = self.m3(x.clone())
        # Perform averaging or any other fusion technique
        x = (x1 + x2 + x3) / 3
        return x

# Initialize ensemble model
ensemble_model = EnsembleModel(m1, m2, m3)

# Print ensemble model summary
print(ensemble_model)


EnsembleModel(
  (m1): VisionTransformer(
    (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (encoder): Encoder(
      (dropout): Dropout(p=0.0, inplace=False)
      (layers): Sequential(
        (encoder_layer_0): EncoderBlock(
          (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (self_attention): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (dropout): Dropout(p=0.0, inplace=False)
          (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): MLPBlock(
            (0): Linear(in_features=768, out_features=3072, bias=True)
            (1): GELU(approximate='none')
            (2): Dropout(p=0.0, inplace=False)
            (3): Linear(in_features=3072, out_features=768, bias=True)
            (4): Dropout(p=0.0, inplace=False)
          )
        )
        (encoder_layer_1): EncoderBlock(
          (ln_1): Laye

In [ ]:

print("Keys in the saved state dictionary:")
print(list(m2_state_dict.keys()))


print("\nKeys in the instantiated ViT model:")
print(list(m2.state_dict().keys()))

Keys in the saved state dictionary:
['base_model.conv1.weight', 'base_model.bn1.weight', 'base_model.bn1.bias', 'base_model.bn1.running_mean', 'base_model.bn1.running_var', 'base_model.bn1.num_batches_tracked', 'base_model.layer1.0.conv1.weight', 'base_model.layer1.0.bn1.weight', 'base_model.layer1.0.bn1.bias', 'base_model.layer1.0.bn1.running_mean', 'base_model.layer1.0.bn1.running_var', 'base_model.layer1.0.bn1.num_batches_tracked', 'base_model.layer1.0.conv2.weight', 'base_model.layer1.0.bn2.weight', 'base_model.layer1.0.bn2.bias', 'base_model.layer1.0.bn2.running_mean', 'base_model.layer1.0.bn2.running_var', 'base_model.layer1.0.bn2.num_batches_tracked', 'base_model.layer1.0.conv3.weight', 'base_model.layer1.0.bn3.weight', 'base_model.layer1.0.bn3.bias', 'base_model.layer1.0.bn3.running_mean', 'base_model.layer1.0.bn3.running_var', 'base_model.layer1.0.bn3.num_batches_tracked', 'base_model.layer1.0.downsample.0.weight', 'base_model.layer1.0.downsample.1.weight', 'base_model.layer1.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


ensemble_model = ensemble_model.to(device)


optimizer = optim.Adam(ensemble_model.parameters())

criterion = nn.CrossEntropyLoss()

epochs = 50

# Training loop
for epoch in range(epochs):
    ensemble_model.train()  # Set the model to train mode

    for inputs, targets in train_dataloader:
        # Move inputs and targets to the appropriate device
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()  # Zero the gradients

        # Forward pass
        outputs = ensemble_model(inputs)

        # Compute the loss
        loss = criterion(outputs, targets)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

    # Validation loop
    ensemble_model.eval()  # Set the model to evaluation mode
    val_correct = 0
    val_total = 0
    true_positives = 0
    predicted_positives = 0
    actual_positives = 0

    with torch.no_grad():
        for inputs, targets in test_dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = ensemble_model(inputs)
            _, predicted = torch.max(outputs, 1)
            val_total += targets.size(0)
            val_correct += (predicted == targets).sum().item()

            # Calculate true positives, predicted positives, and actual positives
            true_positives += ((predicted == 1) & (targets == 1)).sum().item()
            predicted_positives += (predicted == 1).sum().item()
            actual_positives += (targets == 1).sum().item()

    # Calculate precision, recall, and F1-score
    precision = true_positives / predicted_positives if predicted_positives != 0 else 0
    recall = true_positives / actual_positives if actual_positives != 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

    # Calculate validation accuracy
    val_accuracy = val_correct / val_total

    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1_score)
    print(f"Validation Accuracy: {val_accuracy:.4f}")


E:\Anaconda\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Precision: 0.9125
Recall: 0.9125
F1-score: 0.9125
Validation Accuracy: 0.9574
Precision: 0.8837209302325582
Recall: 0.95
F1-score: 0.9156626506024096
Validation Accuracy: 0.9617
Precision: 0.96
Recall: 0.9
F1-score: 0.9290322580645162
Validation Accuracy: 0.9638
Precision: 0.8555555555555555
Recall: 0.9625
F1-score: 0.9058823529411764
Validation Accuracy: 0.9596
Precision: 0.9367088607594937
Recall: 0.925
F1-score: 0.9308176100628932
Validation Accuracy: 0.9681
Precision: 0.9146341463414634
Recall: 0.9375
F1-score: 0.925925925925926
Validation Accuracy: 0.9660
Precision: 0.922077922077922
Recall: 0.8875
F1-score: 0.9044585987261146
Validation Accuracy: 0.9468
Precision: 0.8461538461538461
Recall: 0.9625
F1-score: 0.9005847953216374
Validation Accuracy: 0.9532
Precision: 0.9487179487179487
Recall: 0.925
F1-score: 0.9367088607594937
Validation Accuracy: 0.9638
Precision: 0.926829268292683
Recall: 0.95
F1-score: 0.9382716049382716
Validation Accuracy: 0.9681
Precision: 0.9024390243902439


In [ ]:
import torch
import torch.nn as nn

class WeightedAverageLayer(nn.Module):
    def __init__(self, w1, w2, w3):
        super(WeightedAverageLayer, self).__init__()
        self.w1 = w1
        self.w2 = w2
        self.w3 = w3

    def forward(self, outputs):
        # Perform weighted average computation
        ensemble_output = self.w1 * outputs[0] + self.w2 * outputs[1] + self.w3 * outputs[2]
        return ensemble_output


In [ ]:
import torch.nn as nn

# Define the weighted average layer
class WeightedAverageLayer(nn.Module):
    def __init__(self, w1, w2, w3):
        super(WeightedAverageLayer, self).__init__()
        self.w1 = w1
        self.w2 = w2
        self.w3 = w3

    def forward(self, outputs):
        weighted_sum = self.w1 * outputs[0] + self.w2 * outputs[1] + self.w3 * outputs[2]
        return weighted_sum



w1, w2, w3 = 0.3, 0.1, 0.6
weighted_avg_layer = WeightedAverageLayer(w1, w2, w3)


num_classes = 5

class EnsembleModel(nn.Module):
    def __init__(self, m1, m2, m3, weighted_avg_layer):
        super(EnsembleModel, self).__init__()
        self.model1 = m1
        self.model2 = m2
        self.model3 = m3
        self.weighted_avg_layer = weighted_avg_layer

    def forward(self, x):
        output1 = self.model1(x)
        output2 = self.model2(x)
        output3 = self.model3(x)
        ensemble_output = self.weighted_avg_layer([output1, output2, output3])
        return ensemble_output

# Define ensemble model
ensemble_model = EnsembleModel(pretrained_vit, ResNet50Model(num_classes), DenseNet201Model(num_classes), weighted_avg_layer)

# Check if model has trainable parameters
if list(ensemble_model.parameters()):
    # If there are trainable parameters, define optimizer and loss function
    optimizer = optim.Adam(ensemble_model.parameters())
    criterion = nn.CrossEntropyLoss()
else:
    # If there are no trainable parameters, handle the case appropriately
    print("No trainable parameters found in the model.")

print(ensemble_model)

EnsembleModel(
  (model1): VisionTransformer(
    (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (encoder): Encoder(
      (dropout): Dropout(p=0.0, inplace=False)
      (layers): Sequential(
        (encoder_layer_0): EncoderBlock(
          (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (self_attention): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (dropout): Dropout(p=0.0, inplace=False)
          (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): MLPBlock(
            (0): Linear(in_features=768, out_features=3072, bias=True)
            (1): GELU(approximate='none')
            (2): Dropout(p=0.0, inplace=False)
            (3): Linear(in_features=3072, out_features=768, bias=True)
            (4): Dropout(p=0.0, inplace=False)
          )
        )
        (encoder_layer_1): EncoderBlock(
          (ln_1): 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


ensemble_model.to(device)
num_epochs=30
# Training loop
for epoch in range(num_epochs):
    ensemble_model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, targets in train_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = ensemble_model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        # Accumulate loss
        running_loss += loss.item() * inputs.size(0)

        # Compute accuracy
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    # Calculate average loss and accuracy for the epoch
    epoch_loss = running_loss / len(train_dataloader.dataset)
    epoch_accuracy = correct / total

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Loss: {epoch_loss:.4f}, "
          f"Accuracy: {epoch_accuracy:.4f}")

    # Validation loop
    ensemble_model.eval()  # Set the model to evaluation mode
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, targets in test_dataloader:
            inputs, targets = inputs.to(device), targets.to(device)  # Move data to the same device as the model
            outputs = ensemble_model(inputs)
            _, predicted = torch.max(outputs, 1)
            val_total += targets.size(0)
            val_correct += (predicted == targets).sum().item()

    # Calculate validation accuracy
    val_accuracy = val_correct / val_total

    print(f"Validation Accuracy: {val_accuracy:.4f}")


Epoch [1/30], Loss: 0.0117, Accuracy: 0.9981
Validation Accuracy: 0.9426
Epoch [2/30], Loss: 0.0052, Accuracy: 0.9991
Validation Accuracy: 0.9638
Epoch [3/30], Loss: 0.0046, Accuracy: 0.9981
Validation Accuracy: 0.9511
Epoch [4/30], Loss: 0.0168, Accuracy: 0.9981
Validation Accuracy: 0.9511
Epoch [5/30], Loss: 0.0150, Accuracy: 0.9963
Validation Accuracy: 0.9511
Epoch [6/30], Loss: 0.0186, Accuracy: 0.9963
Validation Accuracy: 0.9574
Epoch [7/30], Loss: 0.0317, Accuracy: 0.9954
Validation Accuracy: 0.9447
Epoch [8/30], Loss: 0.0228, Accuracy: 0.9963
Validation Accuracy: 0.9426
Epoch [9/30], Loss: 0.0033, Accuracy: 1.0000
Validation Accuracy: 0.9553
Epoch [10/30], Loss: 0.0166, Accuracy: 0.9981
Validation Accuracy: 0.9532
Epoch [11/30], Loss: 0.0134, Accuracy: 0.9972
Validation Accuracy: 0.9596
Epoch [12/30], Loss: 0.0089, Accuracy: 0.9981
Validation Accuracy: 0.9574
Epoch [13/30], Loss: 0.0055, Accuracy: 0.9981
Validation Accuracy: 0.9511
Epoch [14/30], Loss: 0.0081, Accuracy: 0.9972
V